# MossFormer2 — Tách Giọng Nói (Speech Separation)
Notebook này chạy trên **Google Colab** để tách 3 nguồn giọng nói từ file audio mix.

## Bước 1: Clone Repository

In [ ]:
!git clone https://github.com/hoanganh0106/Mossformer2.git
%cd Mossformer2

## Bước 2: Cài đặt Dependencies

In [ ]:
!pip install -r requirements.txt -q
!pip install yamlargparse -q
!pip install -e ClearerVoice-Studio-main/clearvoice -q

## Bước 3: Tải Model Checkpoint

Tải file model từ Google Drive và đặt vào đúng thư mục.

In [ ]:
import os
import gdown

ckpt_dir = "ClearerVoice-Studio-main/clearvoice/checkpoints/MossFormer2_SS_16K"
os.makedirs(ckpt_dir, exist_ok=True)

ckpt_path = f"{ckpt_dir}/model.ckpt-61-221600.pt"
if not os.path.exists(ckpt_path):
    print("Đang tải model checkpoint...")
    gdown.download(
        "https://drive.google.com/uc?id=1NfZyzGUQX7g8TMXvO0J6XCmBe098Eu1q",
        ckpt_path,
        quiet=False
    )
    print("✓ Tải xong!")
else:
    print("✓ Checkpoint đã tồn tại.")

## Bước 4: Chọn File Audio Input

Dùng `mix.wav` có sẵn trong repo, hoặc upload file của bạn.

In [ ]:
from google.colab import files

# Lựa chọn: dùng mix.wav có sẵn hoặc upload file mới
USE_DEFAULT = True  # Đổi thành False để upload file của bạn

if USE_DEFAULT:
    INPUT_FILE = "mix.wav"
    print(f"✓ Dùng file mặc định: {INPUT_FILE}")
else:
    print("Upload file audio của bạn:")
    uploaded = files.upload()
    INPUT_FILE = list(uploaded.keys())[0]
    print(f"✓ Đã upload: {INPUT_FILE}")

OUTPUT_DIR = "samples/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Bước 5: Tách Giọng Nói

In [ ]:
import subprocess, sys

input_abs  = os.path.abspath(INPUT_FILE)
output_abs = os.path.abspath(OUTPUT_DIR)

print(f"Input : {input_abs}")
print(f"Output: {output_abs}")

result = subprocess.run(
    [sys.executable, "separate_3spk.py", input_abs, output_abs],
    cwd="ClearerVoice-Studio-main/clearvoice",
    capture_output=True,
    text=True
)

print(result.stdout)
if result.returncode != 0:
    print("=== LỖI ===")
    print(result.stderr)
else:
    print("✓ Hoàn thành!")

## Bước 6: Nghe & Tải Kết Quả

In [ ]:
from IPython.display import Audio, display
from google.colab import files

output_files = sorted([
    os.path.join(OUTPUT_DIR, f)
    for f in os.listdir(OUTPUT_DIR)
    if f.endswith(".wav")
])

if not output_files:
    print("Không tìm thấy file output. Kiểm tra lại bước 5.")
else:
    for i, path in enumerate(output_files, 1):
        print(f"\n🔊 Giọng {i}: {os.path.basename(path)}")
        display(Audio(path))

    print("\n📥 Tải tất cả file output:")
    for path in output_files:
        files.download(path)